In [1]:
# Método alternativo usando pandas directamente
import pandas as pd

# URL en formato CSV (adaptamos la URL del documento)
sheet_id = "1kajUuZwL9l1suipRNy7t2g_SGJUcDVh6MeTu66yK-Z4"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

try:
    # Leer directamente como DataFrame
    df = pd.read_csv(url)
    
    # Mostrar primeros registros
    print(df.head())
except Exception as e:
    print(f"Error al acceder al documento: {e}")
    print("Nota: Este método solo funciona si la hoja es pública o tienes permisos de acceso.")

         Date Session Title       Player Name Split Name      Tags  \
0  15/04/2025   martes 15-4   Orrico Santiago        all  training   
1  15/04/2025   martes 15-4     Espinosa Fede        all  training   
2  15/04/2025   martes 15-4  Rivera Cano Blas        all  training   
3  15/04/2025   martes 15-4  Esquerren Fermin        all  training   
4  15/04/2025   martes 15-4    Urrejola Fabio        all  training   

  Split Start Time Split End Time  Duration  Distance (km)  \
0       45762,8417     45762,9045      5426              6   
1       45762,8383     45762,9316      8058              6   
2        45762,788     45762,9321     12457              7   
3       45762,8383     45762,9329      8175              6   
4       45762,8383     45762,9091      6115              5   

   Sprint Distance (m)  ...  Accelerations Zone Count: 2 - 3 m/s/s  \
0                 2033  ...                                      0   
1                 2108  ...                                      0

In [2]:
df.head(10)

,Date,Session Title,Player Name,Split Name,Tags,Split Start Time,Split End Time,Duration,Distance (km),Sprint Distance (m),...,Accelerations Zone Count: 2 - 3 m/s/s,Accelerations Zone Count: 3 - 4 m/s/s,Accelerations Zone Count: > 4 m/s/s,Deceleration Zone Count: 0 - 1 m/s/s,Deceleration Zone Count: 1 - 2 m/s/s,Deceleration Zone Count: 2 - 3 m/s/s,Deceleration Zone Count: 3 - 4 m/s/s,Deceleration Zone Count: > 4 m/s/s,Unnamed: 98,Unnamed: 99
0,15/04/2025,martes 15-4,Orrico Santiago,all,training,"45762,8417","45762,9045",5426,6,2033,...,0,17,14,38,24,0,19,42,3,27
1,15/04/2025,martes 15-4,Espinosa Fede,all,training,"45762,8383","45762,9316",8058,6,2108,...,0,26,32,37,38,0,41,49,21,14
2,15/04/2025,martes 15-4,Rivera Cano Blas,all,training,"45762,788","45762,9321",12457,7,1142,...,0,30,32,55,15,0,44,42,25,9
3,15/04/2025,martes 15-4,Esquerren Fermin,all,training,"45762,8383","45762,9329",8175,6,1795,...,0,60,29,50,19,0,66,55,20,16
4,15/04/2025,martes 15-4,Urrejola Fabio,all,training,"45762,8383","45762,9091",6115,5,586,...,0,45,32,42,2,0,56,56,9,0
5,15/04/2025,martes 15-4,Lorito,all,training,"45762,8389","45762,9312",7974,6,682,...,0,46,51,26,24,0,87,52,26,6
6,15/04/2025,martes 15-4,Cechi Mateo,all,training,"45762,842","45762,9314",7719,7,2467,...,0,40,29,47,28,0,41,55,19,17
7,15/04/2025,martes 15-4,Fer Fernandez,all,training,"45762,8413","45762,9502",9411,7,2143,...,0,50,27,48,36,0,58,45,20,26
8,22/04/2025,martes 22-4,Espinosa Fede,all,training,"45769,8485","45769,9239",6511,6,1003,...,0,48,48,32,20,0,62,53,23,9
9,22/04/2025,martes 22-4,Orrico Santiago,all,training,"45769,8535","45769,9244",6126,6,1023,...,0,61,43,34,19,0,65,56,23,13


In [3]:
# Importar bibliotecas necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator

# Función para calcular el ratio Agudo:Crónico
def calcular_ratio_AC(dataframe, fecha_columna='Date', distancia_columna='Distance (km)', 
                       agudo_ventana=7, cronico_ventana=28, session_columna='Session Title'):
    """
    Calcula el ratio Agudo:Crónico (AC) basado en la distancia recorrida.
    
    Parámetros:
    - dataframe: DataFrame con los datos
    - fecha_columna: Nombre de la columna con las fechas
    - distancia_columna: Nombre de la columna con las distancias en km
    - agudo_ventana: Ventana para calcular la carga aguda (por defecto: 7 días)
    - cronico_ventana: Ventana para calcular la carga crónica (por defecto: 28 días)
    - session_columna: Nombre de la columna con el título de la sesión/partido
    
    Retorna:
    - DataFrame con fechas, distancias por partido, carga aguda, carga crónica y ratio A:C
    """
    # Asegurarse que la fecha esté en formato datetime
    try:
        if isinstance(dataframe[fecha_columna].iloc[0], str):
            dataframe[fecha_columna] = pd.to_datetime(dataframe[fecha_columna])
    except:
        print("Error al convertir las fechas. Asegúrate que el formato sea correcto.")
        return None
    
    # Agrupar por fecha y partido, sumando las distancias
    df_agrupado = dataframe.groupby([fecha_columna, session_columna])[distancia_columna].sum().reset_index()
    
    # Ordenar por fecha
    df_agrupado = df_agrupado.sort_values(by=fecha_columna)
    
    # Crear una lista para guardar los resultados
    resultados = []
    
    # Para cada fecha a partir de la ventana crónica
    fechas_unicas = sorted(df_agrupado[fecha_columna].unique())
    
    for i in range(cronico_ventana, len(fechas_unicas) + 1):
        fecha_actual = fechas_unicas[i-1]
        
        # Filtrar datos para la fecha actual
        datos_fecha_actual = df_agrupado[df_agrupado[fecha_columna] == fecha_actual]
        
        # Obtener distancia total del partido actual
        distancia_partido = datos_fecha_actual[distancia_columna].sum()
        
        # Datos para ventana aguda (últimos 7 días/partidos)
        indice_inicio_agudo = max(0, i - agudo_ventana)
        fechas_agudo = fechas_unicas[indice_inicio_agudo:i]
        datos_agudo = df_agrupado[df_agrupado[fecha_columna].isin(fechas_agudo)]
        
        # Datos para ventana crónica (últimos 28 días/partidos)
        indice_inicio_cronico = max(0, i - cronico_ventana)
        fechas_cronico = fechas_unicas[indice_inicio_cronico:i]
        datos_cronico = df_agrupado[df_agrupado[fecha_columna].isin(fechas_cronico)]
        
        # Calcular carga aguda y crónica
        carga_aguda = datos_agudo[distancia_columna].mean()
        carga_cronica = datos_cronico[distancia_columna].mean()
        
        # Calcular ratio A:C
        ratio_ac = carga_aguda / carga_cronica if carga_cronica > 0 else 0
        
        # Guardar resultados
        resultados.append({
            'Fecha': fecha_actual,
            'Partido': ', '.join(datos_fecha_actual[session_columna]),
            'Distancia (km)': distancia_partido,
            'Carga Aguda': carga_aguda,
            'Carga Crónica': carga_cronica,
            'Ratio A:C': ratio_ac
        })
    
    # Convertir resultados a DataFrame
    df_resultados = pd.DataFrame(resultados)
    
    return df_resultados

# Función para graficar el ratio A:C
def graficar_ratio_ac(df_resultados):
    """
    Grafica el ratio A:C con anotaciones
    
    Parámetros:
    - df_resultados: DataFrame con los resultados del ratio A:C
    """
    plt.figure(figsize=(12, 8))
    
    # Crear figura principal
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    # Graficar Carga Aguda y Crónica
    ax1.plot(df_resultados['Fecha'], df_resultados['Carga Aguda'], 'b-', label='Carga Aguda (7 días)')
    ax1.plot(df_resultados['Fecha'], df_resultados['Carga Crónica'], 'g-', label='Carga Crónica (28 días)')
    ax1.set_xlabel('Fecha')
    ax1.set_ylabel('Distancia Media (km)', color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    
    # Crear segundo eje Y para el ratio A:C
    ax2 = ax1.twinx()
    ratio_line = ax2.plot(df_resultados['Fecha'], df_resultados['Ratio A:C'], 'r-', label='Ratio A:C')
    ax2.set_ylabel('Ratio A:C', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    # Añadir líneas horizontales para zonas de seguridad
    ax2.axhline(y=0.8, color='orange', linestyle='--', alpha=0.7)
    ax2.axhline(y=1.3, color='orange', linestyle='--', alpha=0.7)
    ax2.fill_between(df_resultados['Fecha'], 0.8, 1.3, alpha=0.1, color='green', label='Zona Óptima (0.8-1.3)')
    
    # Añadir anotaciones para partidos
    for i, row in df_resultados.iterrows():
        if i % 2 == 0:  # Para no saturar el gráfico, anotar cada 2 partidos
            ax1.annotate(f"{row['Partido']}\n{row['Distancia (km)']:.1f}km",
                         (row['Fecha'], row['Carga Aguda']),
                         textcoords="offset points",
                         xytext=(0,10), 
                         ha='center',
                         fontsize=8,
                         arrowprops=dict(arrowstyle='->', color='gray'))
    
    # Configurar formato de fecha en eje X
    date_format = mdates.DateFormatter('%d-%m-%Y')
    ax1.xaxis.set_major_formatter(date_format)
    plt.xticks(rotation=45)
    
    # Unir leyendas de ambos ejes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    plt.title('Evolución del Ratio Agudo:Crónico y Cargas de Trabajo', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    return fig

# Ejecutar el análisis
try:
    # Primero verificar si hay datos y las columnas existen
    if 'df' not in globals() or df is None:
        print("No hay datos cargados o el DataFrame 'df' no existe.")
    else:
        # Verificar columnas necesarias
        columnas_necesarias = ['Date', 'Session Title', 'Distance (km)']
        columnas_faltantes = [col for col in columnas_necesarias if col not in df.columns]
        
        if columnas_faltantes:
            print(f"Faltan las siguientes columnas en el DataFrame: {columnas_faltantes}")
            print("Columnas disponibles:", df.columns.tolist())
        else:
            # Calcular ratio A:C
            df_ratio_ac = calcular_ratio_AC(df)
            
            if df_ratio_ac is not None and not df_ratio_ac.empty:
                # Mostrar tabla de resultados
                print("Tabla de Resultados:")
                print(df_ratio_ac)
                
                # Graficar
                fig = graficar_ratio_ac(df_ratio_ac)
                plt.show()
            else:
                print("No se pudieron calcular los resultados o no hay suficientes datos.")

except Exception as e:
    print(f"Error durante el análisis: {e}")

No se pudieron calcular los resultados o no hay suficientes datos.


C:\Users\dell\AppData\Local\Temp\ipykernel_12152\907127137.py:29: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dataframe[fecha_columna] = pd.to_datetime(dataframe[fecha_columna])


# Cálculo de Promedio de Distancia por Equipo

Esta sección contiene funciones para calcular el promedio de distancia recorrida por el equipo en diferentes sesiones y fechas.

In [4]:
def calcular_promedio_distancia_equipo(df):
    """
    Calcula el promedio de distancia recorrida por el equipo agrupado por fecha y tipo de sesión.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    
    Retorna:
    - DataFrame con los promedios de distancia por fecha y tipo de sesión
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
        
    # Convertir la columna de distancia a numérica (reemplazando comas por puntos)
    # Primero crear una copia para no modificar el df original
    df_dist = df.copy()
    
    try:
        # Convertir la columna 'Distance (km)' a numérica
        if 'Distance (km)' in df_dist.columns:
            # Reemplazar comas por puntos si es necesario
            if isinstance(df_dist['Distance (km)'].iloc[0], str):
                df_dist['Distance (km)'] = df_dist['Distance (km)'].str.replace(',', '.').astype(float)
            elif not pd.api.types.is_numeric_dtype(df_dist['Distance (km)']):
                df_dist['Distance (km)'] = pd.to_numeric(df_dist['Distance (km)'], errors='coerce')
        else:
            print("La columna 'Distance (km)' no existe en el DataFrame.")
            return None
        
        # Convertir la fecha a datetime si es necesario
        if 'Date' in df_dist.columns:
            if not pd.api.types.is_datetime64_dtype(df_dist['Date']):
                df_dist['Date'] = pd.to_datetime(df_dist['Date'], errors='coerce')
        else:
            print("La columna 'Date' no existe en el DataFrame.")
            return None
    
    except Exception as e:
        print(f"Error al convertir los datos: {e}")
        return None
    
    # Calcular el promedio de distancia por fecha y tipo de sesión
    promedio_por_fecha_sesion = df_dist.groupby([df_dist['Date'].dt.date, 'Session Title'])['Distance (km)'].agg(['mean', 'std', 'count']).reset_index()
    promedio_por_fecha_sesion.columns = ['Fecha', 'Tipo de Sesión', 'Promedio Distancia (km)', 'Desviación Estándar (km)', 'Número de Jugadores']
    
    # Calcular el promedio general por tipo de sesión
    promedio_por_sesion = df_dist.groupby('Session Title')['Distance (km)'].agg(['mean', 'std', 'count']).reset_index()
    promedio_por_sesion.columns = ['Tipo de Sesión', 'Promedio Distancia (km)', 'Desviación Estándar (km)', 'Número de Registros']
    
    # Calcular el promedio por jugador
    promedio_por_jugador = df_dist.groupby('Player Name')['Distance (km)'].agg(['mean', 'std', 'count']).reset_index()
    promedio_por_jugador.columns = ['Jugador', 'Promedio Distancia (km)', 'Desviación Estándar (km)', 'Número de Registros']
    promedio_por_jugador = promedio_por_jugador.sort_values(by='Promedio Distancia (km)', ascending=False)
    
    # Crear un diccionario con todos los resultados
    resultados = {
        'promedio_por_fecha_sesion': promedio_por_fecha_sesion,
        'promedio_por_sesion': promedio_por_sesion,
        'promedio_por_jugador': promedio_por_jugador
    }
    
    return resultados

# Ejecutar el cálculo del promedio de distancia
try:
    resultados_distancia = calcular_promedio_distancia_equipo(df)
    
    if resultados_distancia:
        print("Promedio de distancia por tipo de sesión:")
        print(resultados_distancia['promedio_por_sesion'])
        print("\nLos primeros 5 registros del promedio por fecha y tipo de sesión:")
        print(resultados_distancia['promedio_por_fecha_sesion'].head())
        print("\nPromedio de distancia por jugador (Top 10):")
        print(resultados_distancia['promedio_por_jugador'].head(10))
except Exception as e:
    print(f"Error al calcular los promedios: {e}")

Promedio de distancia por tipo de sesión:
       Tipo de Sesión  Promedio Distancia (km)  Desviación Estándar (km)  \
0  Partido San Andres                 6.454545                  1.213560   
1         martes 15-4                 6.250000                  0.707107   
2         martes 22-4                 4.800000                  1.032796   

   Número de Registros  
0                   11  
1                    8  
2                   10  

Los primeros 5 registros del promedio por fecha y tipo de sesión:
        Fecha      Tipo de Sesión  Promedio Distancia (km)  \
0  2025-04-15         martes 15-4                 6.250000   
1  2025-04-22         martes 22-4                 4.800000   
2  2025-04-26  Partido San Andres                 6.454545   

   Desviación Estándar (km)  Número de Jugadores  
0                  0.707107                    8  
1                  1.032796                   10  
2                  1.213560                   11  

Promedio de distancia por jugado

# Visualización de Datos de Distancia

Esta sección presenta visualizaciones de los promedios de distancia recorrida por tipo de sesión y por jugador.

In [5]:
# Visualización de partidos en escala de grises con línea de tendencia
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, IntSlider, HBox, VBox, Output, Layout
import ipywidgets as widgets

def visualizar_solo_partidos(df, max_fechas=None):
    """
    Visualiza datos solo de partidos con esquema de colores negro y gris
    
    Parámetros:
    - df: DataFrame con los datos
    - max_fechas: Número máximo de fechas a mostrar (None para mostrar todas)
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
    
    # Filtrar solo los registros que corresponden a partidos
    partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
    
    if partidos_df.empty:
        print("No se encontraron registros de partidos en los datos.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Convertir distancia a numérica
        if isinstance(partidos_df['Distance (km)'].iloc[0], str):
            partidos_df['Distance (km)'] = partidos_df['Distance (km)'].str.replace(',', '.').astype(float)
        elif not pd.api.types.is_numeric_dtype(partidos_df['Distance (km)']):
            partidos_df['Distance (km)'] = pd.to_numeric(partidos_df['Distance (km)'], errors='coerce')
    
    except Exception as e:
        print(f"Error al convertir datos: {e}")
        return None
    
    # Agrupar por fecha y obtener estadísticas
    partidos_stats = partidos_df.groupby([partidos_df['Date'].dt.date, 'Session Title'])['Distance (km)'].agg(
        ['mean', 'std', 'count', 'sum']).reset_index()
    partidos_stats.columns = ['Fecha', 'Partido', 'Distancia Media (km)', 
                              'Desviación Estándar (km)', 'N° Jugadores', 'Distancia Total (km)']
    
    # Ordenar por fecha ascendente (partidos más antiguos primero)
    partidos_stats = partidos_stats.sort_values('Fecha', ascending=True)
    
    # Limitar por cantidad de fechas si se especifica
    if max_fechas and max_fechas > 0 and max_fechas < len(partidos_stats):
        # Tomar los últimos N partidos (los más recientes)
        partidos_stats = partidos_stats.tail(max_fechas)
    
    return partidos_stats

def generar_visualizacion_interactiva(df):
    """Genera gráficos interactivos para visualizar datos de partidos"""
    
    output = Output()
    
    # Widget para seleccionar número de partidos
    n_partidos_slider = IntSlider(
        value=5, 
        min=1, 
        max=min(20, len(df) if df is not None else 10), 
        step=1, 
        description='N° Partidos:',
        style={'description_width': 'initial'}
    )
    
    # Función para actualizar el gráfico
    def update_graph(n_partidos):
        with output:
            output.clear_output()
            
            if df is None or df.empty:
                print("No hay datos para visualizar")
                return
            
            # Limitar cantidad de partidos a mostrar
            # Tomamos los últimos n_partidos (más recientes)
            df_plot = df.tail(n_partidos).copy()
            
            # Convertir fecha a string para mejor visualización
            df_plot['Fecha'] = df_plot['Fecha'].astype(str)
            
            # Crear figura con Plotly
            fig = go.Figure()
            
            # Añadir línea de tendencia que conecte todos los puntos
            fig.add_trace(go.Scatter(
                x=df_plot['Fecha'] + ' - ' + df_plot['Partido'],
                y=df_plot['Distancia Media (km)'],
                name='Tendencia',
                mode='lines',
                line=dict(color='rgb(30, 30, 30)', width=2, shape='spline'),
                hoverinfo='skip'
            ))
            
            # Añadir barras para distancia media
            fig.add_trace(go.Bar(
                x=df_plot['Fecha'] + ' - ' + df_plot['Partido'],
                y=df_plot['Distancia Media (km)'],
                name='Distancia Media',
                marker_color='rgb(70, 70, 70)',
                text=df_plot['Distancia Media (km)'].round(2),
                textposition='outside',
                hovertemplate='<b>%{x}</b><br>Distancia Media: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total: %{customdata[1]:.2f} km<extra></extra>',
                customdata=df_plot[['N° Jugadores', 'Distancia Total (km)']]
            ))
            
            # Añadir marcadores para cada punto con líneas de error
            fig.add_trace(go.Scatter(
                x=df_plot['Fecha'] + ' - ' + df_plot['Partido'],
                y=df_plot['Distancia Media (km)'],
                error_y=dict(
                    type='data',
                    array=df_plot['Desviación Estándar (km)'],
                    visible=True,
                    color='rgb(20, 20, 20)',
                    thickness=1.5,
                    width=3
                ),
                mode='markers',
                marker=dict(color='rgb(40, 40, 40)', size=8),
                name='Desviación Estándar',
                hovertemplate='<b>%{x}</b><br>Desv. Estándar: %{error_y.array:.2f} km<extra></extra>'
            ))
            
            # Actualizar diseño
            fig.update_layout(
                title='Distancia  por Partido',
                xaxis=dict(
                    title='Fecha y Partido',
                    tickangle=-45
                ),
                yaxis=dict(
                    title='Distancia (km)',
                    gridcolor='rgb(220, 220, 220)'
                ),
                plot_bgcolor='rgb(250, 250, 250)',
                paper_bgcolor='rgb(250, 250, 250)',
                font=dict(color='rgb(20, 20, 20)'),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.1,
                legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                margin=dict(t=80, b=120, l=60, r=40),
                height=600
            )
            
            fig.show()
    
    # Crear la interfaz interactiva
    interact(update_graph, n_partidos=n_partidos_slider)
    
    return VBox([n_partidos_slider, output])

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        partidos_stats = visualizar_solo_partidos(df)
        
        if partidos_stats is not None and not partidos_stats.empty:
            print(f"Se encontraron {len(partidos_stats)} registros de partidos")
            display(generar_visualizacion_interactiva(partidos_stats))
        else:
            print("No se encontraron datos de partidos para visualizar")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    import traceback
    print(f"Error al visualizar los datos: {e}")
    print(traceback.format_exc())

Se encontraron 1 registros de partidos


interactive(children=(IntSlider(value=1, description='N° Partidos:', max=1, min=1, style=SliderStyle(descripti…

In [6]:
# Visualización de partidos en escala de grises - Corregida para mostrar por fecha
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, IntSlider, HBox, VBox, Output, Layout
import ipywidgets as widgets

def visualizar_solo_partidos(df, max_fechas=None):
    """
    Visualiza datos solo de partidos con esquema de colores negro y gris
    
    Parámetros:
    - df: DataFrame con los datos
    - max_fechas: Número máximo de fechas a mostrar (None para mostrar todas)
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
    
    # Filtrar solo los registros que corresponden a partidos
    partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
    
    if partidos_df.empty:
        print("No se encontraron registros de partidos en los datos.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Convertir distancia a numérica
        if isinstance(partidos_df['Distance (km)'].iloc[0], str):
            partidos_df['Distance (km)'] = partidos_df['Distance (km)'].str.replace(',', '.').astype(float)
        elif not pd.api.types.is_numeric_dtype(partidos_df['Distance (km)']):
            partidos_df['Distance (km)'] = pd.to_numeric(partidos_df['Distance (km)'], errors='coerce')
    
    except Exception as e:
        print(f"Error al convertir datos: {e}")
        return None
    
    # Agrupar por fecha y obtener estadísticas
    # IMPORTANTE: Mantenemos la fecha y el título del partido separados para preservar la identidad única
    partidos_stats = partidos_df.groupby([partidos_df['Date'].dt.date, 'Session Title'])['Distance (km)'].agg(
        ['mean', 'std', 'count', 'sum']).reset_index()
    partidos_stats.columns = ['Fecha', 'Partido', 'Distancia Media (km)', 
                              'Desviación Estándar (km)', 'N° Jugadores', 'Distancia Total (km)']
    
    # Ordenar por fecha ascendente (partidos más antiguos primero)
    partidos_stats = partidos_stats.sort_values('Fecha', ascending=True)
    
    # Crear una columna de identificación única que combine fecha y partido
    partidos_stats['Fecha_Partido'] = partidos_stats['Fecha'].astype(str) + ' - ' + partidos_stats['Partido']
    
    # Limitar por cantidad de fechas si se especifica
    if max_fechas and max_fechas > 0 and max_fechas < len(partidos_stats):
        # Tomar los últimos N partidos (los más recientes)
        partidos_stats = partidos_stats.tail(max_fechas)
    
    return partidos_stats

def generar_visualizacion_interactiva(df):
    """Genera gráficos interactivos para visualizar datos de partidos"""
    
    output = Output()
    
    # Widget para seleccionar número de partidos
    n_partidos_slider = IntSlider(
        value=5, 
        min=1, 
        max=min(20, len(df) if df is not None else 10), 
        step=1, 
        description='N° Partidos:',
        style={'description_width': 'initial'}
    )
    
    # Función para actualizar el gráfico
    def update_graph(n_partidos):
        with output:
            output.clear_output()
            
            if df is None or df.empty:
                print("No hay datos para visualizar")
                return
            
            # Limitar cantidad de partidos a mostrar
            # Tomamos los últimos n_partidos (más recientes)
            df_plot = df.tail(n_partidos).copy()
            
            # Crear figura con Plotly
            fig = go.Figure()
            
            # Añadir barras para distancia media
            fig.add_trace(go.Bar(
                x=df_plot['Fecha_Partido'],  # Usamos la combinación única de fecha y partido
                y=df_plot['Distancia Media (km)'],
                name='Distancia Media',
                marker_color='rgb(70, 70, 70)',
                text=df_plot['Distancia Media (km)'].round(2),
                textposition='outside',
                hovertemplate='<b>%{x}</b><br>Distancia Media: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total: %{customdata[1]:.2f} km<extra></extra>',
                customdata=df_plot[['N° Jugadores', 'Distancia Total (km)']]
            ))
            
            # Añadir marcadores para cada punto con líneas de error
            fig.add_trace(go.Scatter(
                x=df_plot['Fecha_Partido'],  # Usamos la combinación única de fecha y partido
                y=df_plot['Distancia Media (km)'],
                error_y=dict(
                    type='data',
                    array=df_plot['Desviación Estándar (km)'],
                    visible=True,
                    color='rgb(20, 20, 20)',
                    thickness=1.5,
                    width=3
                ),
                mode='markers',
                marker=dict(color='rgb(40, 40, 40)', size=8),
                name='Desviación Estándar',
                hovertemplate='<b>%{x}</b><br>Desv. Estándar: %{error_y.array:.2f} km<extra></extra>'
            ))
            
            # Actualizar diseño
            fig.update_layout(
                title='Distancia por Partido',
                xaxis=dict(
                    title='Fecha y Partido',
                    tickangle=-45
                ),
                yaxis=dict(
                    title='Distancia (km)',
                    gridcolor='rgb(220, 220, 220)'
                ),
                plot_bgcolor='rgb(250, 250, 250)',
                paper_bgcolor='rgb(250, 250, 250)',
                font=dict(color='rgb(20, 20, 20)'),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.1,
                legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                margin=dict(t=80, b=120, l=60, r=40),
                height=600
            )
            
            fig.show()
    
    # Crear la interfaz interactiva
    interact(update_graph, n_partidos=n_partidos_slider)
    
    return VBox([n_partidos_slider, output])

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        partidos_stats = visualizar_solo_partidos(df)
        
        if partidos_stats is not None and not partidos_stats.empty:
            print(f"Se encontraron {len(partidos_stats)} registros de partidos")
            display(generar_visualizacion_interactiva(partidos_stats))
        else:
            print("No se encontraron datos de partidos para visualizar")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    import traceback
    print(f"Error al visualizar los datos: {e}")
    print(traceback.format_exc())

Se encontraron 1 registros de partidos


interactive(children=(IntSlider(value=1, description='N° Partidos:', max=1, min=1, style=SliderStyle(descripti…

In [7]:
# Visualización de partidos en escala de grises - Con línea de tendencia y sin desviación estándar
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, IntSlider, HBox, VBox, Output, Layout
import ipywidgets as widgets
import numpy as np

def visualizar_solo_partidos(df, max_fechas=None):
    """
    Visualiza datos solo de partidos con esquema de colores negro y gris
    
    Parámetros:
    - df: DataFrame con los datos
    - max_fechas: Número máximo de fechas a mostrar (None para mostrar todas)
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
    
    # Filtrar solo los registros que corresponden a partidos
    partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
    
    if partidos_df.empty:
        print("No se encontraron registros de partidos en los datos.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Convertir distancia a numérica
        if isinstance(partidos_df['Distance (km)'].iloc[0], str):
            partidos_df['Distance (km)'] = partidos_df['Distance (km)'].str.replace(',', '.').astype(float)
        elif not pd.api.types.is_numeric_dtype(partidos_df['Distance (km)']):
            partidos_df['Distance (km)'] = pd.to_numeric(partidos_df['Distance (km)'], errors='coerce')
    
    except Exception as e:
        print(f"Error al convertir datos: {e}")
        return None
    
    # Agrupar por fecha y obtener estadísticas
    # IMPORTANTE: Mantenemos la fecha y el título del partido separados para preservar la identidad única
    partidos_stats = partidos_df.groupby([partidos_df['Date'].dt.date, 'Session Title'])['Distance (km)'].agg(
        ['mean', 'count', 'sum']).reset_index()  # Eliminamos 'std' del agregado
    partidos_stats.columns = ['Fecha', 'Partido', 'Distancia Media (km)', 
                              'N° Jugadores', 'Distancia Total (km)']
    
    # Ordenar por fecha ascendente (partidos más antiguos primero)
    partidos_stats = partidos_stats.sort_values('Fecha', ascending=True)
    
    # Crear una columna de identificación única que combine fecha y partido
    partidos_stats['Fecha_Partido'] = partidos_stats['Fecha'].astype(str) + ' - ' + partidos_stats['Partido']
    
    # Limitar por cantidad de fechas si se especifica
    if max_fechas and max_fechas > 0 and max_fechas < len(partidos_stats):
        # Tomar los últimos N partidos (los más recientes)
        partidos_stats = partidos_stats.tail(max_fechas)
    
    return partidos_stats

def generar_visualizacion_interactiva(df):
    """Genera gráficos interactivos para visualizar datos de partidos"""
    
    output = Output()
    
    # Widget para seleccionar número de partidos
    n_partidos_slider = IntSlider(
        value=5, 
        min=1, 
        max=min(20, len(df) if df is not None else 10), 
        step=1, 
        description='N° Partidos:',
        style={'description_width': 'initial'}
    )
    
    # Función para actualizar el gráfico
    def update_graph(n_partidos):
        with output:
            output.clear_output()
            
            if df is None or df.empty:
                print("No hay datos para visualizar")
                return
            
            # Limitar cantidad de partidos a mostrar
            # Tomamos los últimos n_partidos (más recientes)
            df_plot = df.tail(n_partidos).copy()
            
            # Crear figura con Plotly
            fig = go.Figure()
            
            # Añadir barras para distancia media
            fig.add_trace(go.Bar(
                x=df_plot['Fecha_Partido'],  # Usamos la combinación única de fecha y partido
                y=df_plot['Distancia Media (km)'],
                name='Distancia Media',
                marker_color='rgb(70, 70, 70)',
                text=df_plot['Distancia Media (km)'].round(2),
                textposition='outside',
                hovertemplate='<b>%{x}</b><br>Distancia Media: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total: %{customdata[1]:.2f} km<extra></extra>',
                customdata=df_plot[['N° Jugadores', 'Distancia Total (km)']]
            ))
            
            # Agregar línea de tendencia que conecte todos los puntos
            fig.add_trace(go.Scatter(
                x=df_plot['Fecha_Partido'],
                y=df_plot['Distancia Media (km)'],
                name='Tendencia',
                mode='lines',
                line=dict(color='rgb(30, 30, 30)', width=3, shape='spline'),
                hovertemplate='<b>%{x}</b><br>Tendencia: %{y:.2f} km<extra></extra>'
            ))
            
            # Actualizar diseño
            fig.update_layout(
                title='Distancia por Partido',
                xaxis=dict(
                    title='Fecha y Partido',
                    tickangle=-45
                ),
                yaxis=dict(
                    title='Distancia (km)',
                    gridcolor='rgb(220, 220, 220)'
                ),
                plot_bgcolor='rgb(250, 250, 250)',
                paper_bgcolor='rgb(250, 250, 250)',
                font=dict(color='rgb(20, 20, 20)'),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.1,
                legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                margin=dict(t=80, b=120, l=60, r=40),
                height=600
            )
            
            fig.show()
    
    # Crear la interfaz interactiva
    interact(update_graph, n_partidos=n_partidos_slider)
    
    return VBox([n_partidos_slider, output])

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        partidos_stats = visualizar_solo_partidos(df)
        
        if partidos_stats is not None and not partidos_stats.empty:
            print(f"Se encontraron {len(partidos_stats)} registros de partidos")
            display(generar_visualizacion_interactiva(partidos_stats))
        else:
            print("No se encontraron datos de partidos para visualizar")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    import traceback
    print(f"Error al visualizar los datos: {e}")
    print(traceback.format_exc())

Se encontraron 1 registros de partidos


interactive(children=(IntSlider(value=1, description='N° Partidos:', max=1, min=1, style=SliderStyle(descripti…

In [8]:
print(df.columns.tolist())

['Date', 'Session Title', 'Player Name', 'Split Name', 'Tags', 'Split Start Time', 'Split End Time', 'Duration', 'Distance (km)', 'Sprint Distance (m)', 'Power Plays', 'Energy (kcal)', 'Impacts', 'Hr Load', 'Time In Red Zone (min)', 'Player Load (estres mecanico)', 'Top Speed (m/s)', 'Distance Per Min (m/min)', 'Power Score (w/kg)', 'Work Ratio', 'Hr Max (bpm)', 'Distance in Speed Zone 1 (km)', 'Distance in Speed Zone 2 (km)', 'Distance in Speed Zone 3 (km)', 'Distance in Speed Zone 4 (km)', 'Distance in Speed Zone 5 (km)', 'Time in Speed Zone 1 (secs)', 'Time in Speed Zone 2 (secs)', 'Time in Speed Zone 3 (secs)', 'Time in Speed Zone 4 (secs)', 'Time in Speed Zone 5 (secs)', 'Impact Zones: 3 - 5 G (Impacts)', 'Impact Zones: 5 - 10 G (Impacts)', 'Impact Zones: 10 - 15 G (Impacts)', 'Impact Zones: 15 - 20 G (Impacts)', 'Impact Zones: > 20 G (Impacts)', 'Power Play Duration Zones: 0 - 2.5 s (Power Plays)', 'Power Play Duration Zones: 2.5 - 5 s (Power Plays)', 'Power Play Duration Zones: 

In [9]:
['Date', 'Session Title', 'Player Name', 'Split Name', 'Tags', 'Split Start Time', 'Split End Time', 'Duration', 'Distance (km)', 'Sprint Distance (m)', 'Power Plays', 'Energy (kcal)', 'Impacts', 'Hr Load', 'Time In Red Zone (min)', 'Player Load (estres mecanico)', 'Top Speed (m/s)', 'Distance Per Min (m/min)', 'Power Score (w/kg)', 'Work Ratio', 'Hr Max (bpm)', 'Distance in Speed Zone 1 (km)', 'Distance in Speed Zone 2 (km)'/, 'Distance in Speed Zone 3 (km)', 'Distance in Speed Zone 4 (km)', 'Distance in Speed Zone 5 (km)', 'Time in Speed Zone 1 (secs)', 'Time in Speed Zone 2 (secs)', 'Time in Speed Zone 3 (secs)', 'Time in Speed Zone 4 (secs)', 'Time in Speed Zone 5 (secs)', 'Impact Zones: 3 - 5 G (Impacts)', 'Impact Zones: 5 - 10 G (Impacts)', 'Impact Zones: 10 - 15 G (Impacts)', 'Impact Zones: 15 - 20 G (Impacts)', 'Impact Zones: > 20 G (Impacts)', 'Power Play Duration Zones: 0 - 2.5 s (Power Plays)', 'Power Play Duration Zones: 2.5 - 5 s (Power Plays)', 'Power Play Duration Zones: 5 - 7.5 s (Power Plays)', 'Power Play Duration Zones: 7.5 - 10 s (Power Plays)', 'Power Play Duration Zones: > 10 s (Power Plays)', 'Distance in Deceleration Zones: 0 - 1 m/s/s (km)', 'Distance in Deceleration Zones: 1 - 2 m/s/s (km)', 'Distance in Deceleration Zones: 2 - 3 m/s/s (km)', 'Distance in Deceleration Zones: 3 - 4 m/s/s (km)', 'Distance in Deceleration Zones: > 4 m/s/s (km)', 'Time in Deceleration Zones: 0 - 1 m/s/s (secs)', 'Time in Deceleration Zones: 1 - 2 m/s/s (secs)', 'Time in Deceleration Zones: 2 - 3 m/s/s (secs)', 'Time in Deceleration Zones: 3 - 4 m/s/s (secs)', 'Time in Deceleration Zones: > 4 m/s/s (secs)', 'Distance in Acceleration Zones: 0 - 1 m/s/s (km)', 'Distance in Acceleration Zones: 1 - 2 m/s/s (km)', 'Distance in Acceleration Zones: 2 - 3 m/s/s (km)', 'Distance in Acceleration Zones: 3 - 4 m/s/s (km)', 'Distance in Acceleration Zones: > 4 m/s/s (km)', 'Time in Acceleration Zones: 0 - 1 m/s/s (secs)', 'Time in Acceleration Zones: 1 - 2 m/s/s (secs)', 'Time in Acceleration Zones: 2 - 3 m/s/s (secs)', 'Time in Acceleration Zones: 3 - 4 m/s/s (secs)', 'Time in Acceleration Zones: > 4 m/s/s (secs)', 'Distance in Power Zone: 0 - 5 w/kg (km)', 'Distance in Power Zone: 5 - 10 w/kg (km)', 'Distance in Power Zone: 10 - 15 w/kg (km)', 'Distance in Power Zone: 15 - 20 w/kg (km)', 'Distance in Power Zone: 20 - 25 w/kg (km)', 'Distance in Power Zone: 25 - 30 w/kg (km)', 'Distance in Power Zone: 30 - 35 w/kg (km)', 'Distance in Power Zone: 35 - 40 w/kg (km)', 'Distance in Power Zone: 40 - 45 w/kg (km)', 'Distance in Power Zone: 45 - 50 w/kg (km)', 'Distance in Power Zone: > 50 w/kg (km)', 'Time in Power Zone: 0 - 5 w/kg (secs)', 'Time in Power Zone: 5 - 10 w/kg (secs)', 'Time in Power Zone: 10 - 15 w/kg (secs)', 'Time in Power Zone: 15 - 20 w/kg (secs)', 'Time in Power Zone: 20 - 25 w/kg (secs)', 'Time in Power Zone: 25 - 30 w/kg (secs)', 'Time in Power Zone: 30 - 35 w/kg (secs)', 'Time in Power Zone: 35 - 40 w/kg (secs)', 'Time in Power Zone: 40 - 45 w/kg (secs)', 'Time in Power Zone: 45 - 50 w/kg (secs)', 'Time in Power Zone: > 50 w/kg (secs)', 'Time in HR Load Zone 0% - 60% Max HR(secs)', 'Time in HR Load Zone 60% - 75% Max HR (secs)', 'Time in HR Load Zone 75% - 85% Max HR (secs)', 'Time in HR Load Zone 85% - 96% Max HR (secs)', 'Time in HR Load Zone 96% - 100% Max HR (secs)', 'Accelerations Zone Count: 0 - 1 m/s/s', 'Accelerations Zone Count: 1 - 2 m/s/s', 'Accelerations Zone Count: 2 - 3 m/s/s', 'Accelerations Zone Count: 3 - 4 m/s/s', 'Accelerations Zone Count: > 4 m/s/s', 'Deceleration Zone Count: 0 - 1 m/s/s', 'Deceleration Zone Count: 1 - 2 m/s/s', 'Deceleration Zone Count: 2 - 3 m/s/s', 'Deceleration Zone Count: 3 - 4 m/ss', 'Deceleration Zone Count: > 4 m/s/s', 'Unnamed: 98', 'Unnamed: 99']

SyntaxError: invalid syntax (2306952459.py, line 1)

In [ ]:
def visualizar_velocidad_equipo(df):
    """
    Visualiza la suma de distancias en zonas de velocidad 3, 4 y 5 dividida por la cantidad de jugadores,
    agrupado por fecha.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Crear una copia del dataframe para no modificar el original
        df_copy = df.copy()
        
        # Convertir fecha a datetime
        if not pd.api.types.is_datetime64_dtype(df_copy['Date']):
            df_copy['Date'] = pd.to_datetime(df_copy['Date'], errors='coerce')
        
        # Convertir columnas de distancia a numéricas
        distance_columns = [
            'Distance in Speed Zone 3 (km)', 
            'Distance in Speed Zone 4 (km)', 
            'Distance in Speed Zone 5 (km)'
        ]
        
        for col in distance_columns:
            if col in df_copy.columns:
                if isinstance(df_copy[col].iloc[0], str):
                    df_copy[col] = df_copy[col].str.replace(',', '.').astype(float)
                elif not pd.api.types.is_numeric_dtype(df_copy[col]):
                    df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
            else:
                print(f"La columna {col} no existe en el DataFrame.")
                return None
    
    except Exception as e:
        print(f"Error al convertir los datos: {e}")
        return None
    
    # Crear una columna con la suma de las distancias en zonas 3, 4 y 5
    df_copy['Suma Zonas Alta Velocidad (km)'] = df_copy[distance_columns].sum(axis=1)
    
    # Agrupar por fecha
    velocidades_por_fecha = df_copy.groupby(df_copy['Date'].dt.date).agg({
        'Suma Zonas Alta Velocidad (km)': 'sum',
        'Player Name': 'nunique'  # Contar jugadores únicos por fecha
    }).reset_index()
    
    # Calcular promedio por jugador
    velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'] = (
        velocidades_por_fecha['Suma Zonas Alta Velocidad (km)'] / 
        velocidades_por_fecha['Player Name']
    )
    
    # Ordenar por fecha (más antiguo primero)
    velocidades_por_fecha = velocidades_por_fecha.sort_values('Date', ascending=True)
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Añadir barras para el promedio por jugador
    fig.add_trace(go.Bar(
        x=velocidades_por_fecha['Date'].astype(str),
        y=velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'],
        text=velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'].round(2),
        textposition='outside',
        marker_color='rgb(70, 70, 70)',
        hovertemplate='<b>%{x}</b><br>Promedio Alta Velocidad: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total Equipo: %{customdata[1]:.2f} km<extra></extra>',
        customdata=velocidades_por_fecha[['Player Name', 'Suma Zonas Alta Velocidad (km)']]
    ))
    
    # Actualizar diseño con los parámetros especificados
    fig.update_layout(
        title='Promedio de Distancia en Alta Velocidad por Jugador',
        xaxis=dict(
            title='Fecha',
            tickangle=-45
        ),
        yaxis=dict(
            title='Distancia (km)',
            gridcolor='rgb(220, 220, 220)'
        ),
        plot_bgcolor='rgb(250, 250, 250)',
        paper_bgcolor='rgb(250, 250, 250)',
        font=dict(color='rgb(20, 20, 20)'),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1,
        showlegend=False,
        margin=dict(t=80, b=120, l=60, r=40),
        height=600
    )
    
    # Mostrar el gráfico
    fig.show()
    
    return velocidades_por_fecha

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        print("Calculando datos de velocidad por equipo...")
        resultados = visualizar_velocidad_equipo(df)
        
        if resultados is not None:
            print(f"Se encontraron datos para {len(resultados)} fechas diferentes")
            print("\nPrimeros registros:")
            print(resultados.head())
        else:
            print("No se pudieron calcular los resultados.")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    import traceback
    print(f"Error al visualizar los datos: {e}")
    print(traceback.format_exc())

Calculando datos de velocidad por equipo...


Se encontraron datos para 24 fechas diferentes

Primeros registros:
         Date  Suma Zonas Alta Velocidad (km)  Player Name  \
0  2021-08-16                            0.45            2   
1  2021-08-17                            2.97            2   
2  2021-08-19                            2.21            2   
3  2021-09-13                            1.26            2   
4  2021-09-14                            7.05            3   

   Promedio Alta Velocidad por Jugador (km)  
0                                     0.225  
1                                     1.485  
2                                     1.105  
3                                     0.630  
4                                     2.350  


In [ ]:
def visualizar_velocidad_equipo(df):
    """
    Visualiza la suma de distancias en zonas de velocidad 3, 4 y 5 dividida por la cantidad de jugadores,
    agrupado por fecha. Muestra las fechas de la más reciente a la más antigua.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Crear una copia del dataframe para no modificar el original
        df_copy = df.copy()
        
        # Convertir fecha a datetime
        if not pd.api.types.is_datetime64_dtype(df_copy['Date']):
            df_copy['Date'] = pd.to_datetime(df_copy['Date'], errors='coerce')
        
        # Convertir columnas de distancia a numéricas
        distance_columns = [
            'Distance in Speed Zone 3 (km)', 
            'Distance in Speed Zone 4 (km)', 
            'Distance in Speed Zone 5 (km)'
        ]
        
        for col in distance_columns:
            if col in df_copy.columns:
                if isinstance(df_copy[col].iloc[0], str):
                    df_copy[col] = df_copy[col].str.replace(',', '.').astype(float)
                elif not pd.api.types.is_numeric_dtype(df_copy[col]):
                    df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
            else:
                print(f"La columna {col} no existe en el DataFrame.")
                return None
    
    except Exception as e:
        print(f"Error al convertir los datos: {e}")
        return None
    
    # Crear una columna con la suma de las distancias en zonas 3, 4 y 5
    df_copy['Suma Zonas Alta Velocidad (km)'] = df_copy[distance_columns].sum(axis=1)
    
    # Agrupar por fecha
    velocidades_por_fecha = df_copy.groupby(df_copy['Date'].dt.date).agg({
        'Suma Zonas Alta Velocidad (km)': 'sum',
        'Player Name': 'nunique'  # Contar jugadores únicos por fecha
    }).reset_index()
    
    # Calcular promedio por jugador
    velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'] = (
        velocidades_por_fecha['Suma Zonas Alta Velocidad (km)'] / 
        velocidades_por_fecha['Player Name']
    )
    
    # Ordenar por fecha descendente (más reciente primero)
    velocidades_por_fecha = velocidades_por_fecha.sort_values('Date', ascending=False)
    
    # Limitar a los últimos 10 registros (que ahora son los más recientes)
    velocidades_por_fecha = velocidades_por_fecha.head(10)
    
    # Invertir el orden para la visualización (más reciente a la derecha)
    velocidades_por_fecha = velocidades_por_fecha.iloc[::-1]
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Añadir barras para el promedio por jugador
    fig.add_trace(go.Bar(
        x=velocidades_por_fecha['Date'].astype(str),
        y=velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'],
        text=velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'].round(2),
        textposition='outside',
        marker_color='rgb(70, 70, 70)',
        hovertemplate='<b>%{x}</b><br>Promedio Alta Velocidad: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total Equipo: %{customdata[1]:.2f} km<extra></extra>',
        customdata=velocidades_por_fecha[['Player Name', 'Suma Zonas Alta Velocidad (km)']]
    ))
    
    # Actualizar diseño con los parámetros especificados
    fig.update_layout(
        title='Promedio de Distancia en Alta Velocidad por Jugador',
        xaxis=dict(
            title='Fecha',
            tickangle=-45
        ),
        yaxis=dict(
            title='Distancia (km)',
            gridcolor='rgb(220, 220, 220)'
        ),
        plot_bgcolor='rgb(250, 250, 250)',
        paper_bgcolor='rgb(250, 250, 250)',
        font=dict(color='rgb(20, 20, 20)'),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1,
        showlegend=False,
        margin=dict(t=80, b=120, l=60, r=40),
        height=600
    )
    
    # Mostrar el gráfico
    fig.show()
    
    return velocidades_por_fecha

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        print("Calculando datos de velocidad por equipo...")
        resultados = visualizar_velocidad_equipo(df)
        
        if resultados is not None:
            print(f"Se muestran los datos de las últimas 10 fechas")
            print("\nRegistros visualizados (de más antigua a más reciente):")
            print(resultados)
        else:
            print("No se pudieron calcular los resultados.")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    import traceback
    print(f"Error al visualizar los datos: {e}")
    print(traceback.format_exc())

Calculando datos de velocidad por equipo...


Se muestran los datos de las últimas 10 fechas

Registros visualizados (de más antigua a más reciente):
          Date  Suma Zonas Alta Velocidad (km)  Player Name  \
14  2022-02-22                          4.7900            1   
15  2022-03-19                          1.7100            1   
16  2022-03-26                          1.6700            1   
17  2022-04-05                          2.7800            1   
18  2022-04-09                          1.2200            1   
19  2022-04-25                          0.6100            1   
20  2022-06-07                          1.4050            1   
21  2022-06-11                          3.2135            1   
22  2022-06-25                          3.8909            1   
23  2023-05-06                         19.7022            1   

    Promedio Alta Velocidad por Jugador (km)  
14                                    4.7900  
15                                    1.7100  
16                                    1.6700  
17            

In [ ]:
def visualizar_velocidad_equipo(df):
    """
    Visualiza la suma de distancias en zonas de velocidad 3, 4 y 5 dividida por la cantidad de jugadores,
    agrupado por fecha. Muestra las fechas de la más reciente a la más antigua.
    Solo considera sesiones de tipo "Partido".
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        print("No hay datos disponibles para analizar.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Crear una copia del dataframe para no modificar el original
        df_copy = df.copy()
        
        # Convertir fecha a datetime
        if not pd.api.types.is_datetime64_dtype(df_copy['Date']):
            df_copy['Date'] = pd.to_datetime(df_copy['Date'], errors='coerce')
        
        # Filtrar solo los registros que corresponden a partidos
        df_copy = df_copy[df_copy['Session Title'].str.contains('Partido', case=False, na=False)]
        
        if df_copy.empty:
            print("No se encontraron registros de partidos en los datos.")
            return None
        
        # Convertir columnas de distancia a numéricas
        distance_columns = [
            'Distance in Speed Zone 3 (km)', 
            'Distance in Speed Zone 4 (km)', 
            'Distance in Speed Zone 5 (km)'
        ]
        
        for col in distance_columns:
            if col in df_copy.columns:
                if isinstance(df_copy[col].iloc[0], str):
                    df_copy[col] = df_copy[col].str.replace(',', '.').astype(float)
                elif not pd.api.types.is_numeric_dtype(df_copy[col]):
                    df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
            else:
                print(f"La columna {col} no existe en el DataFrame.")
                return None
    
    except Exception as e:
        print(f"Error al convertir los datos: {e}")
        return None
    
    # Crear una columna con la suma de las distancias en zonas 3, 4 y 5
    df_copy['Suma Zonas Alta Velocidad (km)'] = df_copy[distance_columns].sum(axis=1)
    
    # Agrupar por fecha
    velocidades_por_fecha = df_copy.groupby(df_copy['Date'].dt.date).agg({
        'Suma Zonas Alta Velocidad (km)': 'sum',
        'Player Name': 'nunique',  # Contar jugadores únicos por fecha
        'Session Title': lambda x: ', '.join(pd.unique(x))  # Guardar los tipos de sesión
    }).reset_index()
    
    # Calcular promedio por jugador
    velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'] = (
        velocidades_por_fecha['Suma Zonas Alta Velocidad (km)'] / 
        velocidades_por_fecha['Player Name']
    )
    
    # Ordenar por fecha descendente (más reciente primero)
    velocidades_por_fecha = velocidades_por_fecha.sort_values('Date', ascending=False)
    
    # Limitar a los últimos 10 registros (que ahora son los más recientes)
    velocidades_por_fecha = velocidades_por_fecha.head(10)
    
    # Invertir el orden para la visualización (más reciente a la derecha)
    velocidades_por_fecha = velocidades_por_fecha.iloc[::-1]
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Añadir barras para el promedio por jugador
    fig.add_trace(go.Bar(
        x=velocidades_por_fecha['Date'].astype(str),
        y=velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'],
        text=velocidades_por_fecha['Promedio Alta Velocidad por Jugador (km)'].round(2),
        textposition='outside',
        marker_color='rgb(70, 70, 70)',
        hovertemplate='<b>%{x}</b><br>Partido: %{customdata[2]}<br>Promedio Alta Velocidad: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total Equipo: %{customdata[1]:.2f} km<extra></extra>',
        customdata=velocidades_por_fecha[['Player Name', 'Suma Zonas Alta Velocidad (km)', 'Session Title']]
    ))
    
    # Actualizar diseño con los parámetros especificados
    fig.update_layout(
        title='Promedio de Distancia en Alta Velocidad por Jugador (Solo Partidos)',
        xaxis=dict(
            title='Fecha',
            tickangle=-45
        ),
        yaxis=dict(
            title='Distancia (km)',
            gridcolor='rgb(220, 220, 220)'
        ),
        plot_bgcolor='rgb(250, 250, 250)',
        paper_bgcolor='rgb(250, 250, 250)',
        font=dict(color='rgb(20, 20, 20)'),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1,
        showlegend=False,
        margin=dict(t=80, b=120, l=60, r=40),
        height=600
    )
    
    # Mostrar el gráfico
    fig.show()
    
    return velocidades_por_fecha

In [ ]:
from ipywidgets import interact, IntSlider, HBox, VBox, Output, Layout
import ipywidgets as widgets

def probar_visualizacion_interactiva(df, max_partidos=10):
    output = Output()
    
    # Widget para seleccionar número de partidos
    n_partidos_slider = IntSlider(
        value=5, 
        min=1, 
        max=max_partidos, 
        step=1, 
        description='N° Partidos:',
        style={'description_width': 'initial'}
    )
    
    # Función para actualizar la visualización
    def update_visualization(n_partidos):
        with output:
            output.clear_output()
            try:
                print(f"Analizando los últimos {n_partidos} partidos...")
                resultados = visualizar_velocidad_equipo(df)
                
                if resultados is not None:
                    subset = resultados.tail(n_partidos)
                    print(f"Promedio total de alta velocidad: {subset['Promedio Alta Velocidad por Jugador (km)'].mean():.2f} km")
            except Exception as e:
                print(f"Error en la visualización: {e}")
    
    # Conectar el widget con la función de actualización
    interact(update_visualization, n_partidos=n_partidos_slider)
    
    return VBox([n_partidos_slider, output])

# Ejecutar la prueba interactiva
if 'df' in globals() and df is not None:
    display(probar_visualizacion_interactiva(df))
else:
    print("DataFrame no disponible. Por favor, cargue los datos primero.")

interactive(children=(IntSlider(value=5, description='N° Partidos:', max=10, min=1, style=SliderStyle(descript…

In [ ]:
# Agrupar por fecha y obtener estadísticas para zonas de velocidad alta
def obtener_estadisticas_velocidad(partidos_df):
    # Primero asegurar que las columnas de velocidad sean numéricas
    columnas_velocidad = [
        'Distance in Speed Zone 3 (km)',
        'Distance in Speed Zone 4 (km)',
        'Distance in Speed Zone 5 (km)'
    ]
    
    # Convertir las columnas a numéricas si es necesario
    for col in columnas_velocidad:
        if isinstance(partidos_df[col].iloc[0], str):
            partidos_df[col] = partidos_df[col].str.replace(',', '.').astype(float)
        elif not pd.api.types.is_numeric_dtype(partidos_df[col]):
            partidos_df[col] = pd.to_numeric(partidos_df[col], errors='coerce')
    
    # Crear columna con la suma de las zonas de velocidad alta
    partidos_df['Alta Velocidad (km)'] = partidos_df[columnas_velocidad].sum(axis=1)
    
    # Agrupar por fecha y obtener estadísticas
    velocidad_stats = partidos_df.groupby([partidos_df['Date'].dt.date, 'Session Title'])['Alta Velocidad (km)'].agg(
        ['mean', 'std', 'count', 'sum']).reset_index()
    
    velocidad_stats.columns = ['Fecha', 'Partido', 'Velocidad Alta Media (km)', 
                              'Desviación Estándar (km)', 'N° Jugadores', 'Velocidad Alta Total (km)']
    
    # Ordenar por fecha
    velocidad_stats = velocidad_stats.sort_values('Fecha', ascending=True)
    
    return velocidad_stats

In [ ]:
try:
    if 'df' in globals() and df is not None:
        # Primero filtrar solo partidos si es necesario
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if not partidos_df.empty:
            # Asegurarse que la fecha está en formato datetime
            if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
                partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
                
            # Obtener estadísticas de velocidad alta
            velocidad_stats = obtener_estadisticas_velocidad(partidos_df)
            
            print(f"Se encontraron datos para {len(velocidad_stats)} partidos")
            print("\nPrimeros 5 registros:")
            print(velocidad_stats.head())
        else:
            print("No se encontraron datos de partidos.")
    else:
        print("No hay datos cargados o el DataFrame 'df' no existe.")
except Exception as e:
    print(f"Error al procesar los datos: {e}")

Se encontraron datos para 10 partidos

Primeros 5 registros:
        Fecha                  Partido  Velocidad Alta Media (km)  \
0  2021-10-16     Partido San Fernando                      1.110   
1  2021-10-16  Partido vs San Fernando                      2.160   
2  2021-10-23        Partido Don Bosco                      0.625   
3  2021-10-30         Partido vs SITAS                      1.545   
4  2022-03-19    partido Liceo Militar                      1.710   

   Desviación Estándar (km)  N° Jugadores  Velocidad Alta Total (km)  
0                       NaN             1                       1.11  
1                       NaN             1                       2.16  
2                  0.247487             2                       1.25  
3                  0.233345             2                       3.09  
4                       NaN             1                       1.71  


C:\Users\dell\AppData\Local\Temp\ipykernel_12412\2991409232.py:9: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [ ]:
try:
    if 'df' in globals() and df is not None:
        # Primero filtrar solo partidos si es necesario
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if not partidos_df.empty:
            # Asegurarse que la fecha está en formato datetime
            if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
                partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
                
            # Obtener estadísticas de velocidad alta
            velocidad_stats = obtener_estadisticas_velocidad(partidos_df)
            
            print(f"Se encontraron datos para {len(velocidad_stats)} partidos")
            print("\nPrimeros 5 registros:")
            print(velocidad_stats.head())
        else:
            print("No se encontraron datos de partidos.")
    else:
        print("No hay datos cargados o el DataFrame 'df' no existe.")
except Exception as e:
    print(f"Error al procesar los datos: {e}")
    

Se encontraron datos para 10 partidos

Primeros 5 registros:
        Fecha                  Partido  Velocidad Alta Media (km)  \
0  2021-10-16     Partido San Fernando                      1.110   
1  2021-10-16  Partido vs San Fernando                      2.160   
2  2021-10-23        Partido Don Bosco                      0.625   
3  2021-10-30         Partido vs SITAS                      1.545   
4  2022-03-19    partido Liceo Militar                      1.710   

   Desviación Estándar (km)  N° Jugadores  Velocidad Alta Total (km)  
0                       NaN             1                       1.11  
1                       NaN             1                       2.16  
2                  0.247487             2                       1.25  
3                  0.233345             2                       3.09  
4                       NaN             1                       1.71  


C:\Users\dell\AppData\Local\Temp\ipykernel_12412\3830013352.py:9: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [ ]:
def obtener_estadisticas_velocidad(partidos_df):
    # Primero asegurar que las columnas de velocidad sean numéricas
    columnas_velocidad = [
        'Distance in Speed Zone 3 (km)',
        'Distance in Speed Zone 4 (km)',
        'Distance in Speed Zone 5 (km)'
    ]
    
    # Convertir las columnas a numéricas si es necesario
    for col in columnas_velocidad:
        if isinstance(partidos_df[col].iloc[0], str):
            partidos_df[col] = partidos_df[col].str.replace(',', '.').astype(float)
        elif not pd.api.types.is_numeric_dtype(partidos_df[col]):
            partidos_df[col] = pd.to_numeric(partidos_df[col], errors='coerce')
    
    # Crear columna con la suma de las zonas de velocidad alta
    partidos_df['Alta Velocidad (km)'] = partidos_df[columnas_velocidad].sum(axis=1)
    
    # Agrupar por fecha y obtener estadísticas
    velocidad_stats = partidos_df.groupby([partidos_df['Date'].dt.date, 'Session Title'])['Alta Velocidad (km)'].agg(
        ['mean', 'std', 'count', 'sum']).reset_index()
    
    velocidad_stats.columns = ['Fecha', 'Partido', 'Velocidad Alta Media (km)', 
                              'Desviación Estándar (km)', 'N° Jugadores', 'Velocidad Alta Total (km)']
    
    # Ordenar por fecha
    velocidad_stats = velocidad_stats.sort_values('Fecha', ascending=True)
    
    return velocidad_stats

In [ ]:
def visualizar_estadisticas_velocidad(df):
    """
    Visualiza las estadísticas de velocidad alta en un gráfico de barras.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    try:
        # Primero filtrar solo partidos
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if partidos_df.empty:
            print("No se encontraron datos de partidos para visualizar.")
            return None
        
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Obtener estadísticas de velocidad
        velocidad_stats = obtener_estadisticas_velocidad(partidos_df)
        
        # Crear identificador único para cada partido (fecha + nombre)
        velocidad_stats['ID_Partido'] = velocidad_stats['Fecha'].astype(str) + ' - ' + velocidad_stats['Partido']
        
        # Limitar a los últimos 10 partidos para mejor visualización
        if len(velocidad_stats) > 10:
            velocidad_stats = velocidad_stats.tail(10)
        
        # Crear gráfico de barras con Plotly
        fig = go.Figure()
        
        # Añadir barras para la velocidad alta media
        fig.add_trace(go.Bar(
            x=velocidad_stats['ID_Partido'],
            y=velocidad_stats['Velocidad Alta Media (km)'],
            name='Velocidad Alta Media',
            marker_color='rgb(70, 70, 70)',
            text=velocidad_stats['Velocidad Alta Media (km)'].round(2),
            textposition='outside',
            hovertemplate='<b>%{x}</b><br>Velocidad Alta Media: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total: %{customdata[1]:.2f} km<extra></extra>',
            customdata=velocidad_stats[['N° Jugadores', 'Velocidad Alta Total (km)']]
        ))
        
        # Añadir línea de tendencia
        fig.add_trace(go.Scatter(
            x=velocidad_stats['ID_Partido'],
            y=velocidad_stats['Velocidad Alta Media (km)'],
            name='Tendencia',
            mode='lines+markers',
            line=dict(color='rgb(30, 30, 30)', width=2),
            marker=dict(color='rgb(30, 30, 30)', size=8),
            hoverinfo='skip'
        ))
        
        # Configurar diseño
        fig.update_layout(
            title='Velocidad Alta Media por Partido',
            xaxis=dict(
                title='Partido',
                tickangle=-45
            ),
            yaxis=dict(
                title='Velocidad Alta Media (km)',
                gridcolor='rgb(220, 220, 220)'
            ),
            plot_bgcolor='rgb(250, 250, 250)',
            paper_bgcolor='rgb(250, 250, 250)',
            font=dict(color='rgb(20, 20, 20)'),
            barmode='group',
            bargap=0.15,
            bargroupgap=0.1,
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
            margin=dict(t=80, b=120, l=60, r=40),
            height=600
        )
        
        # Mostrar gráfico
        fig.show()
        
        return velocidad_stats
    
    except Exception as e:
        import traceback
        print(f"Error al visualizar las estadísticas: {e}")
        print(traceback.format_exc())
        return None

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        resultados = visualizar_estadisticas_velocidad(df)
        if resultados is not None:
            print(f"Se visualizaron datos para {len(resultados)} partidos")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    print(f"Error: {e}")

C:\Users\dell\AppData\Local\Temp\ipykernel_12412\4261746908.py:18: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



Se visualizaron datos para 10 partidos


In [ ]:
def visualizar_estadisticas_velocidad(df):
    """
    Visualiza las estadísticas de velocidad alta en un gráfico de barras.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    try:
        # Primero filtrar solo partidos
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if partidos_df.empty:
            print("No se encontraron datos de partidos para visualizar.")
            return None
        
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Obtener estadísticas de velocidad
        velocidad_stats = obtener_estadisticas_velocidad(partidos_df)
        
        # Limitar a los últimos 10 partidos para mejor visualización
        if len(velocidad_stats) > 10:
            velocidad_stats = velocidad_stats.tail(10)
        
        # Crear gráfico de barras con Plotly
        fig = go.Figure()
        
        # Añadir barras para la velocidad alta media (ahora llamada Sprint)
        fig.add_trace(go.Bar(
            x=velocidad_stats['Partido'],  # Solo mostrar el nombre de la sesión/partido
            y=velocidad_stats['Velocidad Alta Media (km)'],
            name='Sprint',
            marker_color='rgb(70, 70, 70)',
            text=velocidad_stats['Velocidad Alta Media (km)'].round(2),
            textposition='outside',
            hovertemplate='<b>%{x}</b><br>Sprint: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total: %{customdata[1]:.2f} km<extra></extra>',
            customdata=velocidad_stats[['N° Jugadores', 'Velocidad Alta Total (km)']]
        ))
        
        # Añadir línea de tendencia
        fig.add_trace(go.Scatter(
            x=velocidad_stats['Partido'],  # Solo mostrar el nombre de la sesión/partido
            y=velocidad_stats['Velocidad Alta Media (km)'],
            name='Tendencia',
            mode='lines+markers',
            line=dict(color='rgb(30, 30, 30)', width=2),
            marker=dict(color='rgb(30, 30, 30)', size=8),
            hoverinfo='skip'
        ))
        
        # Configurar diseño con nuevo título
        fig.update_layout(
            title='Sprint Por Partido',  # Cambiar a Sprint Por Partido
            xaxis=dict(
                title='Partido',
                tickangle=-45
            ),
            yaxis=dict(
                title='Sprint (km)',  # Actualizado para consistencia
                gridcolor='rgb(220, 220, 220)'
            ),
            plot_bgcolor='rgb(250, 250, 250)',
            paper_bgcolor='rgb(250, 250, 250)',
            font=dict(color='rgb(20, 20, 20)'),
            barmode='group',
            bargap=0.15,
            bargroupgap=0.1,
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
            margin=dict(t=80, b=120, l=60, r=40),
            height=600
        )
        
        # Mostrar gráfico
        fig.show()
        
        return velocidad_stats
    
    except Exception as e:
        import traceback
        print(f"Error al visualizar las estadísticas: {e}")
        print(traceback.format_exc())
        return None

# Ejecutar la visualización
try:
    if 'df' in globals() and df is not None:
        resultados = visualizar_estadisticas_velocidad(df)
        if resultados is not None:
            print(f"Se visualizaron datos para {len(resultados)} partidos")
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    print(f"Error: {e}")


C:\Users\dell\AppData\Local\Temp\ipykernel_12412\4177670914.py:18: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



Se visualizaron datos para 10 partidos


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

def analizar_jugadores_alta_velocidad(df, top_n=10):
    """
    Identifica los jugadores con mayores distancias en zonas de alta velocidad (3, 4 y 5)
    en partidos y muestra un gráfico con los resultados.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    - top_n: Número de jugadores principales a mostrar (default: 10)
    
    Retorna:
    - DataFrame con los jugadores ordenados por distancia total en alta velocidad
    """
    try:
        # Filtrar solo partidos
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if partidos_df.empty:
            print("No se encontraron datos de partidos para analizar.")
            return None
        
        # Columnas de velocidad alta
        columnas_velocidad = [
            'Distance in Speed Zone 3 (km)',
            'Distance in Speed Zone 4 (km)',
            'Distance in Speed Zone 5 (km)'
        ]
        
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Convertir las columnas a numéricas
        for col in columnas_velocidad:
            if isinstance(partidos_df[col].iloc[0], str):
                partidos_df[col] = partidos_df[col].str.replace(',', '.').astype(float)
            elif not pd.api.types.is_numeric_dtype(partidos_df[col]):
                partidos_df[col] = pd.to_numeric(partidos_df[col], errors='coerce')
        
        # Crear columna con la suma total de alta velocidad
        partidos_df['Alta Velocidad Total (km)'] = partidos_df[columnas_velocidad].sum(axis=1)
        
        # También añadir columnas individuales para cada zona
        partidos_df['Zona 3 (km)'] = partidos_df['Distance in Speed Zone 3 (km)']
        partidos_df['Zona 4 (km)'] = partidos_df['Distance in Speed Zone 4 (km)']
        partidos_df['Zona 5 (km)'] = partidos_df['Distance in Speed Zone 5 (km)']
        
        # Agrupar por jugador y calcular estadísticas
        jugadores_stats = partidos_df.groupby('Player Name').agg({
            'Alta Velocidad Total (km)': 'sum',
            'Zona 3 (km)': 'sum',
            'Zona 4 (km)': 'sum', 
            'Zona 5 (km)': 'sum',
            'Session Title': 'count'  # Contar partidos jugados
        }).reset_index()
        
        # Renombrar columna de conteo
        jugadores_stats.rename(columns={'Session Title': 'Partidos Jugados'}, inplace=True)
        
        # Calcular promedio por partido
        jugadores_stats['Promedio por Partido (km)'] = jugadores_stats['Alta Velocidad Total (km)'] / jugadores_stats['Partidos Jugados']
        
        # Ordenar jugadores por distancia total en alta velocidad (descendente)
        jugadores_stats = jugadores_stats.sort_values('Alta Velocidad Total (km)', ascending=False)
        
        # Limitar a los top_n jugadores
        top_jugadores = jugadores_stats.head(top_n)
        
        # Crear gráfico de barras apiladas con Plotly
        fig = go.Figure()
        
        # Añadir barras para cada zona
        fig.add_trace(go.Bar(
            x=top_jugadores['Player Name'],
            y=top_jugadores['Zona 3 (km)'],
            name='Zona 3',
            marker_color='rgb(120, 120, 120)',
            hovertemplate='<b>%{x}</b><br>Zona 3: %{y:.2f} km<extra></extra>'
        ))
        
        fig.add_trace(go.Bar(
            x=top_jugadores['Player Name'],
            y=top_jugadores['Zona 4 (km)'],
            name='Zona 4',
            marker_color='rgb(80, 80, 80)',
            hovertemplate='<b>%{x}</b><br>Zona 4: %{y:.2f} km<extra></extra>'
        ))
        
        fig.add_trace(go.Bar(
            x=top_jugadores['Player Name'],
            y=top_jugadores['Zona 5 (km)'],
            name='Zona 5',
            marker_color='rgb(40, 40, 40)',
            hovertemplate='<b>%{x}</b><br>Zona 5: %{y:.2f} km<extra></extra>'
        ))
        
        # Añadir línea para promedio por partido
        fig.add_trace(go.Scatter(
            x=top_jugadores['Player Name'],
            y=top_jugadores['Promedio por Partido (km)'],
            name='Promedio por Partido',
            mode='lines+markers',
            line=dict(color='rgb(0, 0, 0)', width=2),
            marker=dict(size=8, symbol='circle', color='rgb(0, 0, 0)'),
            yaxis='y2',
            hovertemplate='<b>%{x}</b><br>Promedio por partido: %{y:.2f} km<br>Partidos: %{customdata}<extra></extra>',
            customdata=top_jugadores['Partidos Jugados']
        ))
        
        # Configurar diseño con eje Y secundario para el promedio
        fig.update_layout(
            title='Jugadores con Mayor Distancia en Alta Velocidad (Sprints)',
            xaxis=dict(
                title='Jugador',
                tickangle=-45
            ),
            yaxis=dict(
                title='Distancia Total (km)',
                gridcolor='rgb(220, 220, 220)'
            ),
            yaxis2=dict(
                title='Promedio por Partido (km)',
                overlaying='y',
                side='right',
                showgrid=False
            ),
            plot_bgcolor='rgb(250, 250, 250)',
            paper_bgcolor='rgb(250, 250, 250)',
            font=dict(color='rgb(20, 20, 20)'),
            barmode='stack',
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
            margin=dict(t=80, b=120, l=60, r=80),
            height=600
        )
        
        # Mostrar gráfico
        fig.show()
        
        # También mostrar tabla con datos completos
        print(f"\nTop {top_n} jugadores con mayor distancia en alta velocidad:")
        return jugadores_stats.head(top_n)
    
    except Exception as e:
        import traceback
        print(f"Error al analizar los datos: {e}")
        print(traceback.format_exc())
        return None

# Ejecutar el análisis
try:
    if 'df' in globals() and df is not None:
        top_jugadores = analizar_jugadores_alta_velocidad(df)
        if top_jugadores is not None:
            display(top_jugadores)
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    print(f"Error: {e}")

C:\Users\dell\AppData\Local\Temp\ipykernel_12412\2223414768.py:35: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.




Top 10 jugadores con mayor distancia en alta velocidad:


,Player Name,Alta Velocidad Total (km),Zona 3 (km),Zona 4 (km),Zona 5 (km),Partidos Jugados,Promedio por Partido (km)
3,Duclock Juan,536.3341,279.8916,205.9625,50.4800,134,4.002493
5,Lorito,219.6593,50.3241,110.4553,58.8799,63,3.486656
0,Alejo Vergel,192.9762,112.4859,59.9289,20.5614,52,3.711081
7,Nico Morena,53.8725,12.6843,29.5213,11.6669,14,3.848036
8,Olivero Esteban,3.7742,2.4383,1.2018,0.1341,2,1.887100
13,Wolcan Juan,3.3800,2.3000,0.9400,0.1400,2,1.690000
9,Orrico Santi,2.9158,1.7254,1.0000,0.1904,1,2.915800
2,Astor,2.8546,1.6192,0.9406,0.2948,1,2.854600
12,Urrejola Lisandro,2.6808,2.4931,0.1441,0.0436,2,1.340400
10,Orrico Santiago,1.7100,0.5400,0.7400,0.4300,1,1.710000


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, HBox, VBox, Output, Layout
import ipywidgets as widgets

def analizar_jugadores_por_partido(df, n_partidos=5, top_jugadores=5):
    """
    Analiza los top jugadores con mayores distancias en alta velocidad en los últimos n partidos.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    - n_partidos: Número de partidos recientes a analizar (default: 5)
    - top_jugadores: Número de mejores jugadores a mostrar por partido (default: 5)
    
    Retorna:
    - Lista de DataFrames con los top jugadores para cada partido
    """
    try:
        # Filtrar solo partidos
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if partidos_df.empty:
            print("No se encontraron datos de partidos para analizar.")
            return None
        
        # Columnas de velocidad alta
        columnas_velocidad = [
            'Distance in Speed Zone 3 (km)',
            'Distance in Speed Zone 4 (km)',
            'Distance in Speed Zone 5 (km)'
        ]
        
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Convertir las columnas a numéricas
        for col in columnas_velocidad:
            if isinstance(partidos_df[col].iloc[0], str):
                partidos_df[col] = partidos_df[col].str.replace(',', '.').astype(float)
            elif not pd.api.types.is_numeric_dtype(partidos_df[col]):
                partidos_df[col] = pd.to_numeric(partidos_df[col], errors='coerce')
        
        # Crear columna con la suma total de alta velocidad
        partidos_df['Alta Velocidad (km)'] = partidos_df[columnas_velocidad].sum(axis=1)
        
        # Obtener los últimos n_partidos únicos
        ultimos_partidos = partidos_df.sort_values('Date', ascending=False).drop_duplicates(['Date', 'Session Title']).head(n_partidos)
        fechas_partidos = list(zip(ultimos_partidos['Date'], ultimos_partidos['Session Title']))
        
        # Lista para almacenar resultados
        resultados_partidos = []
        
        # Analizar cada partido por separado
        for fecha, partido in fechas_partidos:
            # Filtrar datos para este partido específico
            partido_data = partidos_df[(partidos_df['Date'] == fecha) & (partidos_df['Session Title'] == partido)]
            
            # Ordenar por distancia en alta velocidad
            partido_data = partido_data.sort_values('Alta Velocidad (km)', ascending=False)
            
            # Seleccionar los top jugadores
            top_data = partido_data.head(top_jugadores)
            
            # Crear DataFrame con info básica
            info_partido = {
                'Fecha': fecha.strftime('%d-%m-%Y'),
                'Partido': partido,
                'Jugadores': len(partido_data),
                'Datos': top_data
            }
            
            resultados_partidos.append(info_partido)
        
        # Visualizar resultados
        visualizar_top_jugadores_por_partido(resultados_partidos)
        
        return resultados_partidos
        
    except Exception as e:
        import traceback
        print(f"Error al analizar los datos: {e}")
        print(traceback.format_exc())
        return None

def visualizar_top_jugadores_por_partido(resultados_partidos):
    """
    Visualiza los mejores jugadores por sprint en cada partido.
    
    Parámetros:
    - resultados_partidos: Lista de diccionarios con la información de cada partido
    """
    # Crear tabs para cada partido
    output = Output()
    
    # Widget para seleccionar el partido
    partido_slider = IntSlider(
        value=0,
        min=0,
        max=len(resultados_partidos)-1,
        step=1,
        description='Partido:',
        style={'description_width': 'initial'}
    )
    
    # Función para actualizar la visualización según el partido seleccionado
    def update_visualization(partido_index):
        with output:
            output.clear_output(wait=True)
            
            if partido_index < 0 or partido_index >= len(resultados_partidos):
                print("Índice de partido fuera de rango")
                return
            
            # Obtener datos del partido
            partido_info = resultados_partidos[partido_index]
            fecha = partido_info['Fecha']
            nombre_partido = partido_info['Partido']
            top_data = partido_info['Datos']
            
            # Crear figura para este partido
            fig = go.Figure()
            
            # Añadir barras para cada zona
            for i, zona in enumerate(['Distance in Speed Zone 3 (km)', 'Distance in Speed Zone 4 (km)', 'Distance in Speed Zone 5 (km)']):
                fig.add_trace(go.Bar(
                    x=top_data['Player Name'],
                    y=top_data[zona],
                    name=f'Zona {i+3}',
                    marker_color=f'rgb({120-i*40}, {120-i*40}, {120-i*40})',
                    hovertemplate='<b>%{x}</b><br>Zona {}: %{y:.2f} km<extra></extra>'.format(i+3)
                ))
            
            # Añadir texto para el total sobre las barras
            for i, jugador in enumerate(top_data['Player Name']):
                total_vel = top_data['Alta Velocidad (km)'].iloc[i]
                fig.add_annotation(
                    x=jugador,
                    y=total_vel,
                    text=f"{total_vel:.2f}",
                    showarrow=False,
                    yshift=10,
                    font=dict(size=10)
                )
            
            # Configurar diseño
            fig.update_layout(
                title=f'Top 5 Jugadores en Sprint - {nombre_partido} ({fecha})',
                xaxis=dict(
                    title='Jugador',
                    tickangle=-45
                ),
                yaxis=dict(
                    title='Distancia (km)',
                    gridcolor='rgb(220, 220, 220)'
                ),
                plot_bgcolor='rgb(250, 250, 250)',
                paper_bgcolor='rgb(250, 250, 250)',
                font=dict(color='rgb(20, 20, 20)'),
                barmode='stack',
                legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                margin=dict(t=80, b=120, l=60, r=60),
                height=600
            )
            
            # Mostrar gráfico
            fig.show()
            
            # Mostrar tabla con datos
            display(top_data[['Player Name', 'Alta Velocidad (km)', 'Distance in Speed Zone 3 (km)', 
                              'Distance in Speed Zone 4 (km)', 'Distance in Speed Zone 5 (km)']])
            
            print(f"\nFecha: {fecha} - {nombre_partido}")
            print(f"Total jugadores en este partido: {partido_info['Jugadores']}")
            
    # Crear la interfaz interactiva
    interact(update_visualization, partido_index=partido_slider)
    
    return VBox([
        widgets.HTML(value="<h3>Top 5 Jugadores por Sprint en los Últimos 5 Partidos</h3>"),
        partido_slider,
        output
    ])

# Ejecutar el análisis
try:
    if 'df' in globals() and df is not None:
        resultados = analizar_jugadores_por_partido(df)
    else:
        print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")
except Exception as e:
    print(f"Error: {e}")

C:\Users\dell\AppData\Local\Temp\ipykernel_12412\3199547640.py:37: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



interactive(children=(IntSlider(value=0, description='Partido:', max=4, style=SliderStyle(description_width='i…

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, VBox, Output
import ipywidgets as widgets

def analizar_top_5_por_partido(df):
    """
    Muestra los 5 jugadores con mayor distancia en alta velocidad para cada uno
    de los últimos 5 partidos, con una interfaz para seleccionar el partido.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    try:
        # Filtrar solo partidos
        partidos_df = df[df['Session Title'].str.contains('Partido', case=False, na=False)].copy()
        
        if partidos_df.empty:
            print("No se encontraron datos de partidos para analizar.")
            return
        
        # Columnas de velocidad alta
        columnas_velocidad = [
            'Distance in Speed Zone 3 (km)',
            'Distance in Speed Zone 4 (km)',
            'Distance in Speed Zone 5 (km)'
        ]
        
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(partidos_df['Date']):
            partidos_df['Date'] = pd.to_datetime(partidos_df['Date'], errors='coerce')
        
        # Convertir las columnas a numéricas
        for col in columnas_velocidad:
            if isinstance(partidos_df[col].iloc[0], str):
                partidos_df[col] = partidos_df[col].str.replace(',', '.').astype(float)
            elif not pd.api.types.is_numeric_dtype(partidos_df[col]):
                partidos_df[col] = pd.to_numeric(partidos_df[col], errors='coerce')
        
        # Crear columna con la suma total de alta velocidad
        partidos_df['Alta Velocidad (km)'] = partidos_df[columnas_velocidad].sum(axis=1)
        
        # Obtener los últimos 5 partidos (fechas distintas)
        ultimos_partidos = partidos_df.sort_values('Date', ascending=False)
        fechas_unicas = ultimos_partidos[['Date', 'Session Title']].drop_duplicates().head(5)
        
        # Crear componentes interactivos
        output = Output()
        
        # Widget para seleccionar partido
        opciones_partidos = [(f"{fecha.strftime('%d-%m-%Y')} - {titulo}", i) 
                            for i, (fecha, titulo) in enumerate(
                                zip(fechas_unicas['Date'], fechas_unicas['Session Title']))]
        
        partido_dropdown = widgets.Dropdown(
            options=opciones_partidos,
            value=0,
            description='Partido:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )
     

C:\Users\dell\AppData\Local\Temp\ipykernel_12412\125759914.py:32: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [ ]:
def procesar_sesiones(df, tipo_sesion='Partido', max_fechas=None):
    """
    Procesa datos de sesiones específicas (Partido, Martes o Jueves)
    
    Parámetros:
    - df: DataFrame con los datos
    - tipo_sesion: Tipo de sesión a procesar ('Partido', 'Martes', 'Jueves')
    - max_fechas: Número máximo de fechas a mostrar (None para mostrar todas)
    """
    # Verificar si el DataFrame es válido
    if df is None or df.empty:
        st.warning("No hay datos disponibles para analizar.")
        return None
    
    # Filtrar solo los registros que corresponden al tipo de sesión
    sesiones_df = df[df['Session Title'].str.contains(tipo_sesion, case=False, na=False)].copy()
    
    if sesiones_df.empty:
        st.warning(f"No se encontraron registros de {tipo_sesion} en los datos.")
        return None
    
    # Convertir columnas necesarias
    try:
        # Convertir fecha a datetime si no lo está
        if not pd.api.types.is_datetime64_dtype(sesiones_df['Date']):
            sesiones_df['Date'] = pd.to_datetime(sesiones_df['Date'], errors='coerce')
        
        # Convertir distancia a numérica
        if isinstance(sesiones_df['Distance (km)'].iloc[0], str):
            sesiones_df['Distance (km)'] = sesiones_df['Distance (km)'].str.replace(',', '.').astype(float)
        elif not pd.api.types.is_numeric_dtype(sesiones_df['Distance (km)']):
            sesiones_df['Distance (km)'] = pd.to_numeric(sesiones_df['Distance (km)'], errors='coerce')
    
    except Exception as e:
        st.error(f"Error al convertir datos: {e}")
        return None
    
    # Agrupar por fecha y obtener estadísticas
    sesiones_stats = sesiones_df.groupby([sesiones_df['Date'].dt.date, 'Session Title'])['Distance (km)'].agg(
        ['mean', 'count', 'sum']).reset_index()
    
    sesiones_stats.columns = ['Fecha', 'Sesión', 'Distancia Media (km)', 
                              'N° Jugadores', 'Distancia Total (km)']
    
    # Ordenar por fecha ascendente
    sesiones_stats = sesiones_stats.sort_values('Fecha', ascending=True)
    
    # Crear una columna de identificación única que combine fecha y sesión
    sesiones_stats['Fecha_Sesion'] = sesiones_stats['Sesión']  # Solo usar el título de la sesión.astype(str) + ' - ' + sesiones_stats['Sesión']
    
    # Limitar por cantidad de fechas si se especifica
    if max_fechas and max_fechas > 0 and max_fechas < len(sesiones_stats):
        # Tomar los últimos N registros (los más recientes)
        sesiones_stats = sesiones_stats.tail(max_fechas)
    
    return sesiones_stats

def graficar_distancia_sesion(sesiones_stats, n_sesiones, tipo_sesion):
    """
    Genera el gráfico de distancia por sesiones
    """
    if sesiones_stats is None or sesiones_stats.empty:
        st.warning("No hay datos para visualizar")
        return
    
    # Limitar cantidad de sesiones a mostrar
    # Tomamos las últimas n_sesiones (más recientes)
    df_plot = sesiones_stats.tail(n_sesiones).copy()
    
    # Crear figura con Plotly
    fig = go.Figure()
    
    # Añadir barras para distancia media
    fig.add_trace(go.Bar(
        x=df_plot['Fecha_Sesion'],
        y=df_plot['Distancia Media (km)'],
        name='Distancia Media',
        marker_color='rgb(70, 70, 70)',
        text=df_plot['Distancia Media (km)'].round(2),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Distancia Media: %{y:.2f} km<br>N° Jugadores: %{customdata[0]}<br>Total: %{customdata[1]:.2f} km<extra></extra>',
        customdata=df_plot[['N° Jugadores', 'Distancia Total (km)']]
    ))
    
    # Agregar línea de tendencia
    fig.add_trace(go.Scatter(
        x=df_plot['Fecha_Sesion'],
        y=df_plot['Distancia Media (km)'],
        name='Tendencia',
        mode='lines',
        line=dict(color='rgb(30, 30, 30)', width=3, shape='spline'),
        hovertemplate='<b>%{x}</b><br>Tendencia: %{y:.2f} km<extra></extra>'
    ))
    
    # Actualizar diseño
    fig.update_layout(
    title={
        'text': f'Distancia en (km) - {tipo_sesion}',
        'font': {'size': 24},
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title='Sesión',
        tickangle=-30,  # Ángulo menos pronunciado
        tickfont=dict(size=14),  # Texto más grande
        title_font=dict(size=16)  # Título más grande
    ),
    yaxis=dict(
        title='Distancia (km)',
        gridcolor='rgb(220, 220, 220)'
    ),
        plot_bgcolor='rgb(250, 250, 250)',
        paper_bgcolor='rgb(250, 250, 250)',
        font=dict(color='rgb(20, 20, 20)'),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        height=500
    )
    
    return fig

In [ ]:
# Definir los tipos de sesiones a analizar
tipos_sesion = ['Partido', 'Martes', 'Jueves']

# Procesar y mostrar datos para cada tipo de sesión
for tipo_sesion in tipos_sesion:
    print(f"\n=== Análisis de sesiones: {tipo_sesion} ===")
    
    # Procesar datos de la sesión
    resultados = procesar_sesiones(df, tipo_sesion=tipo_sesion, max_fechas=10)
    
    if resultados is not None:
        print(f"\nÚltimas 10 sesiones de {tipo_sesion}:")
        print("\nEstadísticas principales:")
        print(f"Total de sesiones encontradas: {len(resultados)}")
        print(f"Promedio de distancia por sesión: {resultados['Distancia Media (km)'].mean():.2f} km")
        print(f"Promedio de jugadores por sesión: {resultados['N° Jugadores'].mean():.1f}")
        
        # Mostrar tabla con los datos
        print("\nDetalle de las últimas sesiones:")
        display(resultados[['Fecha', 'Sesión', 'Distancia Media (km)', 'N° Jugadores', 'Distancia Total (km)']])
        
        # Crear y mostrar el gráfico
        fig = graficar_distancia_sesion(resultados, n_sesiones=10, tipo_sesion=tipo_sesion)
        fig.show()
    else:
        print(f"No se encontraron datos para sesiones de {tipo_sesion}")


=== Análisis de sesiones: Partido ===

Últimas 10 sesiones de Partido:

Estadísticas principales:
Total de sesiones encontradas: 10
Promedio de distancia por sesión: 5401.20 km
Promedio de jugadores por sesión: 9.3

Detalle de las últimas sesiones:


C:\Users\dell\AppData\Local\Temp\ipykernel_13968\1502734827.py:26: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,Fecha,Sesión,Distancia Media (km),N° Jugadores,Distancia Total (km)
30,2024-05-18,Partido Banco Nacion 18/5,5.000000,10,50.0
31,2024-05-25,Partido Italiano,4.600000,10,46.0
32,2024-06-01,Partido Cuq,3.625000,8,29.0
33,2024-06-08,Partido Moreno,4.200000,10,42.0
34,2024-06-22,Partido Liceo Naval,2.000000,10,20.0
35,2024-06-29,Partido Don Bosco,4.800000,10,48.0
36,2024-07-06,Partido Liceo Militiar,4.000000,10,40.0
37,2024-07-14,Partido San Carlos,4.300000,10,43.0
38,2024-08-10,Partido Delta11/8,8.000000,4,32.0
39,2025-04-26,Partido San Andres,53971.454545,11,593686.0



=== Análisis de sesiones: Martes ===

Últimas 10 sesiones de Martes:

Estadísticas principales:
Total de sesiones encontradas: 5
Promedio de distancia por sesión: 3.72 km
Promedio de jugadores por sesión: 3.4

Detalle de las últimas sesiones:


,Fecha,Sesión,Distancia Media (km),N° Jugadores,Distancia Total (km)
0,2022-01-02,Pretemporada Martes,5.48,1,5.48
1,2022-07-02,Pretemporada Martes,5.38,1,5.38
2,2022-07-06,Entrenamiento Martes,2.00,3,6.00
3,2023-04-04,Entrenamiento Martes,3.25,4,13.00
4,2024-06-08,martes 6/8,2.50,8,20.00



=== Análisis de sesiones: Jueves ===

Últimas 10 sesiones de Jueves:

Estadísticas principales:
Total de sesiones encontradas: 3
Promedio de distancia por sesión: 4.19 km
Promedio de jugadores por sesión: 1.7

Detalle de las últimas sesiones:


,Fecha,Sesión,Distancia Media (km),N° Jugadores,Distancia Total (km)
0,2022-10-02,Pretemporada Jueves,5.900000,1,5.9
1,2023-02-03,Entrenamiento Jueves,4.000000,1,4.0
2,2023-06-07,Entrenamiento Jueves 6/7,2.666667,3,8.0


In [10]:
def analizar_velocidad_maxima(df):
    """
    Analiza y visualiza la velocidad máxima de los jugadores por sesión.
    
    Parámetros:
    - df: DataFrame con los datos de Catapult
    """
    try:
        # Crear una copia del DataFrame
        df_speed = df.copy()
        
        # Convertir Top Speed de m/s a km/h
        df_speed['Top Speed (km/h)'] = df_speed['Top Speed (m/s)'] * 3.6
        
        # Agrupar por sesión y obtener el jugador más rápido
        top_speeds = df_speed.groupby(['Session Title', 'Player Name'])['Top Speed (km/h)'].max().reset_index()
        
        # Ordenar por velocidad descendente dentro de cada sesión
        top_speeds = top_speeds.sort_values(['Session Title', 'Top Speed (km/h)'], ascending=[True, False])
        
        # Crear figura con Plotly
        fig = go.Figure()
        
        # Obtener sesiones únicas
        sesiones = top_speeds['Session Title'].unique()
        
        # Crear barras para cada sesión
        for sesion in sesiones:
            datos_sesion = top_speeds[top_speeds['Session Title'] == sesion]
            # Tomar los 5 jugadores más rápidos de cada sesión
            top_5 = datos_sesion.head(5)
            
            fig.add_trace(go.Bar(
                name=sesion,
                x=top_5['Player Name'],
                y=top_5['Top Speed (km/h)'],
                text=top_5['Top Speed (km/h)'].round(1),
                textposition='outside',
                hovertemplate='<b>%{x}</b><br>' +
                            'Velocidad Máxima: %{y:.1f} km/h<br>' +
                            'Sesión: ' + sesion +
                            '<extra></extra>'
            ))
        
        # Actualizar diseño
        fig.update_layout(
            title={
                'text': 'Top 5 Velocidades Máximas por Sesión',
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            xaxis_title='Jugador',
            yaxis_title='Velocidad Máxima (km/h)',
            barmode='group',
            plot_bgcolor='rgb(250, 250, 250)',
            paper_bgcolor='rgb(250, 250, 250)',
            font=dict(size=12),
            showlegend=True,
            legend_title='Sesión',
            height=600
        )
        
        # Mostrar gráfico
        fig.show()
        
        # Mostrar tabla con resultados
        print("\nVelocidades máximas por sesión:")
        for sesion in sesiones:
            print(f"\n=== {sesion} ===")
            datos_sesion = top_speeds[top_speeds['Session Title'] == sesion]
            print(datos_sesion.head().to_string(index=False))
            
        return top_speeds
        
    except Exception as e:
        print(f"Error al analizar las velocidades: {e}")
        return None

# Ejecutar el análisis
if 'df' in globals() and df is not None:
    resultados = analizar_velocidad_maxima(df)
else:
    print("No se encontraron datos cargados. Ejecute primero la celda para cargar los datos.")


Velocidades máximas por sesión:

=== Partido San Andres ===
     Session Title         Player Name  Top Speed (km/h)
Partido San Andres              Lorito             118.8
Partido San Andres         Cechi Mateo             115.2
Partido San Andres    Rivera Cano Blas             115.2
Partido San Andres     Orrico Santiago             108.0
Partido San Andres Portunato Sebastian             108.0

=== martes 15-4 ===
Session Title      Player Name  Top Speed (km/h)
  martes 15-4    Espinosa Fede             104.4
  martes 15-4      Cechi Mateo             100.8
  martes 15-4    Fer Fernandez             100.8
  martes 15-4 Esquerren Fermin              97.2
  martes 15-4  Orrico Santiago              97.2

=== martes 22-4 ===
Session Title     Player Name  Top Speed (km/h)
  martes 22-4     Cechi Mateo             108.0
  martes 22-4    Vergel Alejo             108.0
  martes 22-4   Espinosa Fede             104.4
  martes 22-4 Orrico Santiago             104.4
  martes 22-4        

In [12]:
# Filtrar el último partido o sesión
ultima_sesion = df['Session Title'].iloc[-1]
ultima_fecha = df['Date'].iloc[-1]

# Filtrar los datos para la última sesión
df_ultima_sesion = df[
    (df['Session Title'] == ultima_sesion) & 
    (df['Date'] == ultima_fecha)
].copy()

# Convertir Top Speed de m/s a km/h 
df_ultima_sesion['Top Speed (km/h)'] = df_ultima_sesion['Top Speed (m/s)'] * 3.6

# Ordenar por velocidad ascendente
velocidades_ordenadas = df_ultima_sesion[['Player Name', 'Top Speed (km/h)']] \
    .sort_values('Top Speed (km/h)', ascending=True)

# Crear gráfico con Plotly
fig = go.Figure()

# Añadir barras para velocidades
fig.add_trace(go.Bar(
    x=velocidades_ordenadas['Player Name'],
    y=velocidades_ordenadas['Top Speed (km/h)'],
    text=velocidades_ordenadas['Top Speed (km/h)'].round(1),
    textposition='outside',
    marker_color='rgb(70, 70, 70)',
    hovertemplate='<b>%{x}</b><br>Velocidad Máxima: %{y:.1f} km/h<extra></extra>'
))

# Actualizar diseño
fig.update_layout(
    title=f'Velocidades Máximas - {ultima_sesion}',
    xaxis=dict(
        title='Jugador',
        tickangle=-45
    ),
    yaxis=dict(
        title='Velocidad Máxima (km/h)',
        gridcolor='rgb(220, 220, 220)'
    ),
    plot_bgcolor='rgb(250, 250, 250)',
    paper_bgcolor='rgb(250, 250, 250)',
    showlegend=False,
    height=600
)

# Mostrar gráfico
fig.show()

# Mostrar tabla de resultados
display(velocidades_ordenadas)

,Player Name,Top Speed (km/h)
28,Damioli Bautista,86.4
21,Fer Fernandez,100.8
18,Franco Suarez,104.4
19,Olivero Esteban,104.4
25,Cachaza Pato,104.4
26,Nicolas Morena,104.4
20,Orrico Santiago,108.0
27,Portunato Sebastian,108.0
23,Rivera Cano Blas,115.2
24,Cechi Mateo,115.2


In [23]:
# Ordenar por velocidad ascendente
velocidades_ordenadas = df_ultima_sesion[['Player Name', 'Top Speed (m/s)']] \
    .sort_values('Top Speed (m/s)', ascending=True)

# Crear gráfico con Plotly
fig = go.Figure()

# Añadir barras para velocidades
fig.add_trace(go.Bar(
    x=velocidades_ordenadas['Player Name'],
    y=velocidades_ordenadas['Top Speed (m/s)'],
    text=velocidades_ordenadas['Top Speed (m/s)'].round(1),
    textposition='outside',
    marker_color='rgb(70, 70, 70)',
    hovertemplate='<b>%{x}</b><br>Velocidad Máxima: %{y:.1f} m/s<extra></extra>'
))

# Actualizar diseño
fig.update_layout(
    title=f'Velocidades Máximas - {ultima_sesion}',
    xaxis=dict(
        title='Jugador',
        tickangle=-45
    ),
    yaxis=dict(
        title='Velocidad Máxima (m/s)',
        gridcolor='rgb(220, 220, 220)'
    ),
    plot_bgcolor='rgb(250, 250, 250)',
    paper_bgcolor='rgb(250, 250, 250)',
    showlegend=False,
    height=600
)

# Mostrar gráfico
fig.show()

# Mostrar tabla de resultados
display(velocidades_ordenadas)

,Player Name,Top Speed (m/s)
28,Damioli Bautista,24
21,Fer Fernandez,28
18,Franco Suarez,29
19,Olivero Esteban,29
25,Cachaza Pato,29
26,Nicolas Morena,29
20,Orrico Santiago,30
27,Portunato Sebastian,30
23,Rivera Cano Blas,32
24,Cechi Mateo,32
